In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

In [7]:
def model(X, w, w2, w3, w4, w_o, p_keep_conv, p_keep_hidden):
    l1a = tf.nn.relu(tf.nn.conv2d(X, w,                       # l1a shape=(?, 48, 48, 32)
                        strides=[1, 1, 1, 1], padding='SAME'))
    l1 = tf.nn.max_pool(l1a, ksize=[1, 2, 2, 1],              # l1 shape=(?, 24, 24, 32)
                        strides=[1, 2, 2, 1], padding='SAME')
    l1 = tf.nn.dropout(l1, p_keep_conv)

    l2a = tf.nn.relu(tf.nn.conv2d(l1, w2,                     # l2a shape=(?, 24, 24, 64)
                        strides=[1, 1, 1, 1], padding='SAME'))
    l2 = tf.nn.max_pool(l2a, ksize=[1, 2, 2, 1],              # l2 shape=(?, 12, 12, 64)
                        strides=[1, 2, 2, 1], padding='SAME')
    l2 = tf.nn.dropout(l2, p_keep_conv)

    l3a = tf.nn.relu(tf.nn.conv2d(l2, w3,                     # l3a shape=(?, 12, 12, 128)
                        strides=[1, 1, 1, 1], padding='SAME'))
    l3 = tf.nn.max_pool(l3a, ksize=[1, 2, 2, 1],              # l3 shape=(?, 6, 6, 128)
                        strides=[1, 2, 2, 1], padding='SAME')
    l3 = tf.reshape(l3, [-1, w4.get_shape().as_list()[0]])    # reshape to (?, 4608)
    l3 = tf.nn.dropout(l3, p_keep_conv)

    l4 = tf.nn.relu(tf.matmul(l3, w4))
    l4 = tf.nn.dropout(l4, p_keep_hidden)

    pyx = tf.matmul(l4, w_o)
    return pyx

In [8]:
arr = []

with open("./binary_cnn_data.csv", "r") as f:
    for line in f:
        arr.append(list(map(int, line.split(','))))


arr = np.array(arr)
arr.shape

(2622, 6914)

In [9]:
X = tf.placeholder("float", [None, 48, 48, 3])
Y = tf.placeholder("float", [None, 2])

w = init_weights([3, 3, 3, 32])       # 3x3x3 conv, 32 outputs
w2 = init_weights([3, 3, 32, 64])     # 3x3x32 conv, 64 outputs
w3 = init_weights([3, 3, 64, 128])    # 3x3x64 conv, 128 outputs
w4 = init_weights([4608, 625])        # FC 4608 inputs, 625 outputs
w_o = init_weights([625, 2])          # FC 625 inputs, 2 outputs (labels)

p_keep_conv = tf.placeholder("float")
p_keep_hidden = tf.placeholder("float")

training_size = 1835
image_size = 6912

In [10]:
py_x = model(X, w, w2, w3, w4, w_o, p_keep_conv, p_keep_hidden)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=Y))
train_op = tf.train.AdamOptimizer(0.001, 0.9).minimize(cost)
predict_op = tf.argmax(py_x, 1)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(100):
        np.random.shuffle(arr)
        trX = arr[:training_size, :image_size].reshape((-1, 48, 48, 3))
        trY = arr[:training_size, image_size:]
        
        sess.run(train_op, feed_dict={X: trX, Y: trY,
                                      p_keep_conv: 0.8, p_keep_hidden: 0.5})

        
        teX = arr[training_size:, :image_size].reshape((-1, 48, 48, 3))
        teY = arr[training_size:, image_size:]
        print(i, np.mean(np.argmax(teY, axis=1) ==
                         sess.run(predict_op, feed_dict={X: teX,
                                                         Y: teY,
                                                         p_keep_conv: 1.0,
                                                         p_keep_hidden: 1.0})))



0 0.5374841168996188
1 0.5692503176620076
2 0.41296060991105465
3 0.4269377382465057
4 0.44472681067344344
5 0.43964421855146124
6 0.4599745870393901
7 0.43456162642947904
8 0.4409148665819568
9 0.4409148665819568
10 0.4434561626429479
11 0.42947903430749684
12 0.4498094027954257
13 0.44599745870393903
14 0.4371029224904701
15 0.5133418043202033
16 0.6048284625158831
17 0.7153748411689962
18 0.6200762388818297
19 0.6035578144853876
20 0.5895806861499364
21 0.6569250317662008
22 0.6925031766200762
23 0.6645489199491741
24 0.6797966963151207
25 0.7102922490470139
26 0.6925031766200762
27 0.7280813214739518
28 0.7255400254129606
29 0.7229987293519695
30 0.761118170266836
31 0.7052096569250318
32 0.7407878017789072
33 0.6696315120711563
34 0.7102922490470139
35 0.7255400254129606
36 0.6658195679796697
37 0.6645489199491741
38 0.7128335451080051
39 0.7598475222363406
40 0.7128335451080051
41 0.7102922490470139
42 0.7229987293519695
43 0.761118170266836
44 0.6874205844980941
45 0.68996188055